In [1]:
import torch
from torchvision import models
from torchvision import transforms
import time

In [2]:
model = models.quantization.inception_v3(
    weights=True, quantize=True)


/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_QuantizedWeights.IMAGENET1K_FBGEMM_V1`. You can also use `weights=Inception_V3_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torch/ao/quantization/utils.py:280: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [3]:
model.eval()

QuantizableInception3(
  (Conv2d_1a_3x3): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(3, 32, kernel_size=(3, 3), stride=(2, 2), scale=0.04087992012500763, zero_point=0)
    (bn): Identity()
    (relu): Identity()
  )
  (Conv2d_2a_3x3): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(32, 32, kernel_size=(3, 3), stride=(1, 1), scale=0.044250644743442535, zero_point=2)
    (bn): Identity()
    (relu): Identity()
  )
  (Conv2d_2b_3x3): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(32, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.046114157885313034, zero_point=0, padding=(1, 1))
    (bn): Identity()
    (relu): Identity()
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(64, 80, kernel_size=(1, 1), stride=(1, 1), scale=0.04493110254406929, zero_point=0)
    (bn): Identity()
    (relu): Identity()
  )
  (Conv2d_4a_3x3): QuantizableBasicConv2d(


In [3]:
# Load the image, resize it and convert it to a tensor and display it
from PIL import Image

img = Image.open('cat.jpg')
img = transforms.Resize((299, 299))(img)
img = transforms.ToTensor()(img)



In [4]:
# Run the model on the image
start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Time taken: ', end - start)

Time taken:  0.04535412788391113


In [5]:
# Running inference 100 times
start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')


Time: 1.8932514190673828 seconds


In previous notebook, single image inference took 0.0921 sec.
Now its 0.0162 sec

Around 5 faster

For the 100 times inference, it took 3.670 sec earlier
After quantization, it took 1.139 sec

Around 3.2x faster

## Fusing layers

https://spell.ml/blog/pytorch-quantization-X8e7wBAAACIAHPhT

This is a really good article that goes over pytorch model quantization, it also goes over fusing layers, which improves the performance of the model, when used in python.